## Manifold Methods for Visualisation of High-dimensional Data
 
**Data** [Compositional Data](2_Data_CompositionalData.ipynb) | [Lambdas](2_Data_Lambdas.ipynb) **Classifiers** [Building Models](3_Classifiers_ClassifierModels.ipynb) | [Feature Selection](3_Classifiers_Features.ipynb) **Visualisation**  [Entropy](4_Vis_Entropy.ipynb) | [Manifolds](4_Vis_Manifolds.ipynb) **pyrolite** [Overview](6_pyrolite.ipynb) **GitHub** [AEGC2019](https://github.com/morganjwilliams/aegc2019) | [pyrolite](https://github.com/morganjwilliams/pyrolite)


Visualisation of data which has high dimensionality is challenging, and one solution is to provide visualisations in low-dimension representations of the space actually spanned by the data. Here we provide an example of visualisation of classification predictions and relative prediction certainty for a dataset from [Ueki2018] (using entropy across predicted probability for each individual class, [adapted from pyrolite documentation](https://pyrolite.readthedocs.io/en/develop/usage/eg/plotting/manifold.html), and also explained in a separate notebook [here](4_Vis_Entropy.ipynb)).

[Ueki2018]: https://doi.org/10.1029/2017GC007401 "Ueki, K., Hino, H., Kuwatani, T., 2018. Geochemical Discrimination and Characteristics of Magmatic Tectonic Settings: A Machine-Learning-Based Approach. Geochemistry, Geophysics, Geosystems 19, 1327–1347."

We'll import some standard packages, build a classifer and try to construct a visualiation combining both an approximation of the high-dimensional spatial relationships (those being utilised by the classifier) in addition to information about the probability uncertainty (where more faded colors indicate less certain classifications).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from pyrolite.util.skl.pipeline import SVC_pipeline
from pyrolite.util.skl.vis import plot_mapping
from pyrolite.util.plot import __DEFAULT_DISC_COLORMAP__

In [ ]:
from pyrolite.util.skl.transform import CLRTransform
df = pd.read_csv('https://storage.googleapis.com/aegc2019/ueki2018.csv')
classes = {c: ix for ix, c in enumerate(df.Class.unique())}
X, y = CLRTransform().transform(df.loc[:, [i for i in df.columns if i != "Class"]]), df["Class"]
y = y.apply(lambda x: classes[x]) # turn these into integers for the colormap

In [ ]:
svc = SVC_pipeline(probability=True, scaler=StandardScaler(), kernel='rbf')
gs = svc.fit(X, y)

In [ ]:
from pyrolite.util.plot import proxy_line

fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 4))

xsample = X.sample(500)
s=40

a, tfm, mapped = plot_mapping(
    xsample, 
    gs.best_estimator_, 
    ax=ax[1], 
    s=s,
    init="pca"
)
ax[0].scatter(*mapped.T, c=__DEFAULT_DISC_COLORMAP__(gs.predict(xsample)), s=s)

ax[0].set_xticks([])
ax[0].set_yticks([])

ax[1].legend([proxy_line(c=__DEFAULT_DISC_COLORMAP__(ix), ls='none', marker='o') for c, ix in classes.items()], 
             [c for c in classes.keys()], 
             bbox_to_anchor=(1.05, 1.0), 
             loc='upper left', 
             frameon=False, 
             fontsize=12);

In [ ]:
from pyrolite.util.plot import save_figure
save_figure(fig, 
            name='ManifoldMappingEntropy', 
            save_at="../presentation/figures/", 
            save_fmts=['pdf', 'png'])

### UMAP

Uniform Manifold Approximation and Projection (UMAP) is a dimension reduction technique which can be used in a very similar way to other manifold methods within scikit-learn ([McInnes2018]), with the added flexibility of being able transform new/novel data. This is particulary useful for visualising training and testing data together, but can also be applied to map reference compositions and boundaries into the projected space. UMAP can also be used for semi-supervised and supervised dimensional reduction and metric learning, but this is beyond the scope of these notebooks.

The technique is based on a specific set of assumptions which may be violated in practise, but it remains effective for the uses described here.


[McInnes2018]:  https://doi.org/10.21105/joss.00861 "McInnes, L., Healy, J., Saul, N., Grossberger, L., 2018. UMAP: uniform manifold approximation and projection. The Journal of Open Source Software 3, 861."

In [ ]:
import umap.umap_ as umap  # required in case of umap install errors

reducer = umap.UMAP(metric='mahalanobis', min_dist=0.1, n_neighbors=30)
embedding = reducer.fit_transform(X)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
ax.scatter(*embedding.T, 
           c=__DEFAULT_DISC_COLORMAP__(gs.predict(X)),
           s=s)
ax.set_xticks([])
ax.set_yticks([])
ax.legend([proxy_line(c=__DEFAULT_DISC_COLORMAP__(ix), ls='none', marker='o') for c, ix in classes.items()], 
          [c for c in classes.keys()],
          bbox_to_anchor=(1.05, 1.0), 
          loc='upper left', 
          frameon=False, 
          fontsize=12);

In [ ]:
from pyrolite.util.plot import save_figure
save_figure(fig, 
            name='UMAP', 
            save_at="../presentation/figures/", 
            save_fmts=['pdf', 'png'])

You can also change the number of components, for example to generate 3D figures:

In [ ]:
threeDreducer = umap.UMAP(n_components=3, metric='mahalanobis', n_neighbors=50)
threeDembedding = threeDreducer.fit_transform(X)

In [ ]:
from pyrolite.util.skl import alphas_from_multiclass_prob

c = __DEFAULT_DISC_COLORMAP__(gs.predict(X)) # predicted classes
ps = gs.predict_proba(X) # probabilities
c[:, -1] = alphas_from_multiclass_prob(ps, alpha=0.95) # append alphas to RGBA colours

In [ ]:
import ipyvolume as ipv
ipv.figure()
ipv.quickscatter(*threeDembedding.T,
                 color=c,
                 size=2, 
                 marker='sphere')

In [ ]:
import time
import base64

def change_angle(fig, n, frac):
    with fig:
        ipv.view(azimuth=frac*360)
        
time.sleep(3) # make sure to wait until the figure is properly rendered before trying to capture output
tmpdir = ipv.movie('test.gif', 
                   change_angle,
                   cmd_template_gif="magick convert -delay {delay} {loop} {tempdir}/frame-*.png {filename}",
                   fps=10, frames=100, endpoint=False)

with open('test.gif', 'rb') as gif:
    gifurl = b"data:image/gif;base64," +base64.b64encode(gif.read())

In [ ]:
import IPython.display
IPython.display.Image(url=gifurl.decode('ascii'))

In [ ]:
# from IPython.display import Video
# IPython.display.Video("test.mp4")

In [ ]:
# https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic_multinomial.html